In [1]:
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
import torch
import pandas as pd
import os
import importlib
import nglview

_ColormakerRegistry()

In [2]:
%load_ext autoreload
%autoreload 2

###  Conformation Generation

In [2]:
import DataGen
from DataGen import genconfs
from DataGen import util
from DataGen.genconfs import runGenerator
from DataGen.util import get_RMSD

The main function here is runGenerator

In [7]:
help(genconfs.runGenerator)

Help on function runGenerator in module DataGen.genconfs:

runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir=None, numConfs=300, clusterMethod='RMSD', clusterThreshold=0.2)
    Generate conformation as sdf for all smiles in input
    index_list: the list of initial indexes from source data 
    smiles_list: the rdkit smiles generated using initial source data SMILES
    source_data_name: the source data name, such as ccdc, zinc, and pubchem, which is used to record the initial index in that source data
    datadir: directory for outputfile
    structure_dir: the directory used to find the 3D structure file, if it is None, the numConfs should not be None
    numConfs: the number of conformations generated before clustering, defaults to 300; if numConfs=None, we conducted local MMFF minimization for structure
    clusterMethod: the distance calculation method used in clustering, defaults to RMSD
    clusterThreshold: the clustering threshold, defaults to 0.2


For variables such as clusterMethod and clusterThreshold,, we can always use default values

#### Conformation generation for smiles with no structure reference

In [8]:
### I use the first molecule in Index_ccdc_20_overlapwithmol20_removeempty.csv ###
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = None
numConfs = 300
%mkdir "../test/confs_smiles"
datadir = "../test/confs_smiles"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

mkdir: ../test/confs_smiles: File exists
0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
The total number of conformers after clustring: 254


In [9]:
Failed_list

[]

After conformation genertaion, the generated conformation(254 conformations in this case) has been saved in a 33_confors.sdf in ../test/confs_smiles. <br>The return of this function is the list for all failed cases (idx of index_list), there it's empty since no failed case.

#### Conformation generation for smiles with structure reference

This time we use reference structure of this SMILES from ccdc dataset.<br>
The reason to do this is that we want to make sure all newly generated conformations have right protonation state.<br>
When we have crystal structure, we should use crystal structure as reference to generate conformations

In [8]:
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = "../test/reference_structure"
numConfs = 300
%mkdir "../test/confs_smiles_withreference"
datadir = "../test/confs_smiles_withreference"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
Use 3D Structure as Reference: 33.sdf
The total number of conformers after clustring: 248


#### Local optimization for reference structure

To conduct the local minimization, we don't need conformation generation, and hence the numConfs should be 0 

In [32]:
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = "../test/reference_structure"
numConfs = 0
%mkdir "../test/confs_local_minimization"
datadir = "../test/confs_local_minimization"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
Use 3D Structure as Reference: 33.sdf
Finish Local Minimization with MMFF


#### Check generated conformations

In [3]:
### Check Attribute
conf_1 = PandasTools.LoadSDF("../test/confs_smiles/33_confors.sdf")
conf_2 = PandasTools.LoadSDF("../test/confs_smiles_withreference/33_confors.sdf")
conf_1.head()
conf_2.head()

,energy_abs,SMILES,cluster_no,ccdc_id,initial_conformation_id,minimize_method,ID,ROMol
0,44.5737028374007,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,1,33,283,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
1,44.82607464748033,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,2,33,258,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
2,46.0972404834002,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,3,33,235,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
3,44.54444482507678,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,4,33,214,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
4,43.55545027087747,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,5,33,154,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."


,energy_abs,SMILES,cluster_no,ccdc_id,initial_conformation_id,minimize_method,ID,ROMol
0,44.925354442201694,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,1,33,260,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
1,43.905396003013806,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,2,33,254,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
2,44.835954486888966,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,3,33,191,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
3,44.16119342798035,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,4,33,278,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
4,50.24077902861522,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,5,33,274,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."


In [4]:
### Check RMSD
reference_1 = Chem.SDMolSupplier("../test/reference_structure/33.sdf")
reference_2 = Chem.SDMolSupplier("../test/confs_local_minimization/33_min.sdf")
prob_1 = Chem.SDMolSupplier("../test/confs_smiles/33_confors.sdf")
prob_2 = Chem.SDMolSupplier("../test/confs_smiles_withreference/33_confors.sdf")
get_RMSD(reference_1[0], reference_2[0])
get_RMSD(prob_2[0], reference_2[0])
get_RMSD(prob_1[0], reference_2[0])

0.1510568782980715

1.1748416601353016

1.323696229379208

In [5]:
### View Structure: can only show the first one

nglview.show_file("../test/confs_smiles/33_confors.sdf")
nglview.show_file("../test/confs_smiles_withreference/33_confors.sdf")

NGLWidget()

NGLWidget()

### Generate Gaussian Input

In [4]:
import DataGen
from DataGen import genGaus
from DataGen.genGaus import file_seperate
from DataGen.genGaus import file_transfer
from DataGen.genGaus import gaussian_gen

#### Generate gaussian input for local minimized structure ( sdf file with only one conformation)

In [5]:
index_list = ["33"]
### if ftype is cry --> *_min.sdf; else: *.sdf 
ftype = "cry"
header_file = "../test/confs_local_minimization/header.txt"
datadir = "../test/confs_local_minimization/"
file_transfer(index_list, ftype, datadir)
gaussian_gen(index_list, header_file, datadir)

In [7]:
### Check the output
ginput = open("../test/confs_local_minimization/33.opt.com").readlines()
ginput

['%NProcShared=1\n',
 '%Mem=10GB\n',
 '\n',
 '#p B3LYP/6-31G* opt freq\n',
 '\n',
 'test\n',
 '\n',
 '0  1\n',
 'O           2.98360         2.50210        13.57860\n',
 'O           3.70340         2.09020        15.70480\n',
 'O           1.49550         1.85930         7.39580\n',
 'C           2.80590         2.35140        14.92170\n',
 'C           1.35720         2.58870        15.26940\n',
 'H           1.27470         3.65490        15.52350\n',
 'C           0.64780         2.31320        13.91390\n',
 'C           1.73970         2.91640        12.99030\n',
 'H           1.73780         4.01420        13.07000\n',
 'C           0.91260         1.77830        16.47680\n',
 'H          -0.17200         1.82760        16.61180\n',
 'H           1.37880         2.17440        17.38630\n',
 'H           1.21720         0.72890        16.40730\n',
 'C          -0.65520         3.11850        13.84360\n',
 'H          -1.37150         2.77230        14.59580\n',
 'H          -1.127

#### Generate gaussian input for conformations

In [9]:
infile_list = ["33_confors.sdf"]
datadir = "../test/confs_smiles/"
%mkdir "../test/confs_smiles/ginput"
outdir = "../test/confs_smiles/ginput/"
### use this initial index, we can reorder all conformations just based on conformation orders
initial_index = 0
### if initial_index = 0 , the conformations will start from 1 
final_index = file_seperate(infile_list, datadir, outdir, initial_index)

In [10]:
final_index
### this will be next initail_index for next molecule

254

In [11]:
index_list = [i for i in range(1, 255)]
### if ftype is cry --> *_min.sdf; else: *.sdf 
ftype = "conformations"
header_file = "../test/confs_local_minimization/header.txt"
datadir = "../test/confs_smiles/ginput/"
file_transfer(index_list, ftype, datadir)
gaussian_gen(index_list, header_file, datadir)

**Warning** The file transfer is based on obabel, and it takes some time to finish, and don't do this transfermation for so many conformations at the same time on your own computer, it can cause problem 

In [12]:
### Check the output
ginput = open("../test/confs_smiles/ginput/1.opt.com").readlines()
ginput

['%NProcShared=1\n',
 '%Mem=10GB\n',
 '\n',
 '#p B3LYP/6-31G* opt freq\n',
 '\n',
 'test\n',
 '\n',
 '0  1\n',
 'C           4.98580         2.46560         3.31510\n',
 'O           4.14670         1.34220         3.55200\n',
 'C           3.04200         1.21860         2.75580\n',
 'C           2.23710         0.11500         3.04030\n',
 'C           1.07590        -0.13240         2.30290\n',
 'C           0.69750         0.72070         1.25690\n',
 'C          -0.55890         0.43040         0.45960\n',
 'O          -1.08480         1.62800        -0.12300\n',
 'C          -1.84580         1.29440        -1.20000\n',
 'O          -2.53450         2.09250        -1.81480\n',
 'C          -1.72100        -0.18350        -1.49790\n',
 'C          -1.72730        -0.40950        -3.00500\n',
 'C          -0.44480        -0.57960        -0.72150\n',
 'C           0.84100        -0.32340        -1.53630\n',
 'C          -0.48370        -2.02980        -0.15170\n',
 'C          -0.822

### Generate Jobs for HPC

In [1]:
import DataGen
from DataGen import genjob_hpc
from DataGen.genjob_hpc import gen_job
from DataGen.genjob_hpc import sub_job
from DataGen.genjob_hpc import check_gaussian

#### Generate jobs

In [4]:
index_list = [i for i in range(1,255)]
### the number of jobs that have been generated 
current_jobs = 20
index_list
%mkdir "../test/confs_smiles/gjobs"
jobdir = "../test/confs_smiles/gjobs"
datadir = "/beegfs/jl7003/1Dto3D_DataGen/test/confs_smiles/ginput"
### the number of calculations in each job
num_jobs=10
### time requried for each job
time=5
### memory required for each job
mem=10
gen_job(index_list, current_jobs, jobdir, datadir,  num_jobs=num_jobs, time=time, mem=mem)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

mkdir: ../test/confs_smiles/gjobs: File exists


Try different time, memory combination.<br>
Notice: in HPC, job has small time and memory requirement don't need to wait for lone time. For large molecule, such as molecule with 20 heavy atoms, you need only contain one molecule in the job. For small molecule, such as molecule with 10 heavy atoms, you can make more than one molecule in the job.

#### Submit jobs

In [6]:
### Submit jobs
job_list = [i for i in range(1,24)]
jobdir = "../test/confs_smiles/gjobs"
### job range is determined by id1 and id2 
id1 = 0
id2 = 2
sub_job(job_list, id1, id2, jobdir)

In HPC, we can at most submit 1000 jobs, but only 200 jobs can run at the same time.

#### Check output: error or normal

In [3]:
help(check_gaussian)

Help on function check_gaussian in module DataGen.genjob_hpc:

check_gaussian(index_list, datadir)
    Check whether the gaussian calculation finishes normally



In [4]:
### Here, we used the log file from 33_min.sdf as an example
### The file is 33.opt.log in ../test/confs_local_minimization
### Gaussian_error.csv will be saved in current dir
index_list = ["33"]
datadir = "../test/confs_local_minimization/"
check_gaussian(index_list, datadir)

In [5]:
error = open("../test/confs_local_minimization/Gaussian_error.csv").readline()

In [6]:
error

''

Since our calculation finishes normally, there is no record in error file 

### Check Calculations Results

In [2]:
import DataGen
from DataGen.analysis import getInfor
from DataGen.analysis import check

Here, we conducted consistency check, to see if our initial SMILES, MMFF optimized structure, and QM optimized structure are consistent. <br>
We used files in *test/test_analysis* as examples. There are five calculations, however, the QM calculation for index 3 didn't finish normally, thus, our index_list only includes other indexes.

In [3]:
help(check)

Help on class check in module DataGen.analysis:

class check(builtins.object)
 |  check(datadir, index_list, outdir, confs=False)
 |  
 |  Analysis data after QM calculations
 |  
 |  Methods defined here:
 |  
 |  __get_relation__(self)
 |      Get initial index list and smiles list for each calculated file
 |  
 |  __init__(self, datadir, index_list, outdir, confs=False)
 |      :param datadir: directory for all data files including MMFF optimized sdf file and QM calculated log file
 |      :param outdir: directory for all output data index files 
 |      :param index_list: index_list for all successful calculated files. Exp. 1.opt.log, 2.opt.log --> index_list[1,2]
 |      :param confs: if calculated files are from different conformations, consf=True, else, False. Defaults to False
 |  
 |  built_initialdata(self)
 |      Build initial information data
 |  
 |  check_consistency(self, rule)
 |      Check consistency of initial data, MMFF optimized data, and QM optimized data. 
 |   

In [4]:
datadir = "../test/test_analysis/"
index_list = [1,2,4,5]
outdir = "../test/test_analysis/index_files" ### directory for output index files
confs = True ### if conducted calculations for conformations, should be True. Defaults to False
ana_check = check(datadir, index_list, outdir, confs=True)

In [5]:
### basic information ###
ana_check.index_list
ana_check.index_init_list ### initial index, which is the index before conformation generation ###
ana_check.smiles_list ### initial SMILES ####

[1, 2, 4, 5]

['0', '1', '3', '4']

['B(C1=CC=CC1)C1=CC=CC1',
 'B(C1CCCC1)C1CCCC1',
 'B(OC1CC1)c1ccccc1',
 'B(c1cccs1)c1cccs1']

For different conformations generated from same initial molecule, the initial index and SMILES should be same. <br>
Here, I used four conformations generated from four different initial molecules

#### Build Initial Index Data

We built initial index data which includes all informations we need. During this process, opt.log file has been converted into opt.sdf file.

In [6]:
### initial index file will be saved into outdir with name as initial_dataset.csv
ana_check.built_initialdata()

In [7]:
initial_data = pd.read_csv(os.path.join(outdir, "initial_dataset.csv"), sep = " ")
initial_data

,index,initial_index,SMILES,initial_InChI,initial_SMILES,MMFF_InChI,MMFF_SMILES,QM_InChI,QM_SMILES
0,1,0,B(C1=CC=CC1)C1=CC=CC1,InChI=1S/C10H11B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1=CC=CC1)C1=CC=CC1,InChI=1S/C10H11B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1=CC=CC1)C1=CC=CC1,InChI=1S/C10H10B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(=C1[CH]C=CC1)C1=CC=CC1
1,2,1,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1
2,4,3,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1
3,5,4,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1


Here, SMILES is the SMILES we our initial SMILES. <br>
initial_SMILES is the SMILES converted from SMILES, and should be same as SMILES. <br>
MMFF_SMILES is the SMILES for MMFF optimized structure. <br>
QM_SMILES is the SMILES for QM optimized structure. <br>

#### Consistency Check 

Based on initial_dataset.csv, we first conducted consistency check.<br>
Here, **rule = "strict"** means we will only keep the structures with same initial_SMILES, MMFF_SMILES, and QM_SMILES;
**rule = "loose"** means for structure with different SMILES, we further check its InChIs, if InChIs are same, we keep these structures.

In [9]:
### defaults output will be data_consistent_strict.csv ####
rule = "strict"
ana_check.check_consistency(rule)
data_strict = pd.read_csv(os.path.join(outdir, "data_consistent_strict.csv"))
data_strict

,index,initial_index,SMILES,initial_InChI,initial_SMILES,MMFF_InChI,MMFF_SMILES,QM_InChI,QM_SMILES
0,2,1,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1
1,4,3,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1
2,5,4,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1


Above table shows that structure with index = 1 has been removed since its MMFF_SMILES is not same as QM_SMILES

In [10]:
### defaults output will be data_consistent_loose.csv ####
rule = "loose"
ana_check.check_consistency(rule)
data_strict = pd.read_csv(os.path.join(outdir, "data_consistent_loose.csv"))
data_strict

,index,initial_index,SMILES,initial_InChI,initial_SMILES,MMFF_InChI,MMFF_SMILES,QM_InChI,QM_SMILES
0,2,1,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1
1,4,3,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1
2,5,4,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1


Above table shows that structure with index = 1 has been removed since its MMFF_SMILES is not same as QM_SMILES, and its MMFF_InChI is not same as QM_InChI.

#### Radical Check and Partial Charge Check

Since all our initial molecules have no partial charges and radicals, thus, if 3D structure has partial charges or radicals, it should be removed.
Here, we only need to check MMFF optimized structure, since we already finished the consistency check and QM optimized structure is converted from coordinates, which can be wrong.

In [12]:
infile = "data_consistent_strict.csv" ### file generated after consistency check
ana_check.check_others(infile)
### default output file will be infile.split(".")[0] + "_rmrpc.csv" ###
data_strict_rmrpc = pd.read_csv(os.path.join(outdir, infile.split(".")[0] + "_rmrpc.csv"))
data_strict_rmrpc

,index,initial_index,SMILES,initial_InChI,initial_SMILES,MMFF_InChI,MMFF_SMILES,QM_InChI,QM_SMILES
0,2,1,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1,InChI=1S/C10H19B/c1-2-6-9(5-1)11-10-7-3-4-8-10...,B(C1CCCC1)C1CCCC1
1,4,3,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1,InChI=1S/C9H11BO/c1-2-4-8(5-3-1)10-11-9-6-7-9/...,B(OC1CC1)c1ccccc1
2,5,4,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1,InChI=1S/C8H7BS2/c1-3-7(10-5-1)9-8-4-2-6-11-8/...,B(c1cccs1)c1cccs1


There's nothing change, because all three structures have no partial charges or radicals in their MMFF_SMILES

### Prepare dataset

In [2]:
import DataGen
from DataGen import prepare_data
from DataGen.prepare_data import data_infor
from DataGen.prepare_data import prepre_PhysNet_input
from DataGen.prepare_data import prepare_torch
from DataGen.prepare_data import prepare_target_csv

#### Get data infor

In [3]:
datadir = "../test/confs_local_minimization/"
reference = "../test/reference_method/atomref.B3LYP_631Gd.10As.npz"
index_list = ["33"]
ftype = "cry"
exp = data_infor(index_list[0], datadir, reference, ftype)

data_infor returns a object with all informations you need to prepare a dataset. 

In [4]:
### All attributes you can get from this object, detailed description can be found in source code 
[ i for i in dir(exp) if i[0] != "_"]

['MMFFcoords',
 'MMFFlines',
 'MMFFmol',
 'MMFFsdf',
 'QMcoords',
 'QMlines',
 'QMmol',
 'QMsdf',
 'charges_mulliken',
 'conversions',
 'datadir',
 'dipole',
 'elementdict',
 'elementdict_periodic',
 'elements',
 'elements_p',
 'get_coordinates',
 'get_dipole',
 'get_elements',
 'get_initial_target',
 'get_mulliken_charges',
 'get_target',
 'index',
 'init_target',
 'log',
 'loglines',
 'natoms',
 'reference',
 'target',
 'target_names']

#### Prepare dataset

In [5]:
### prepare torch file to save rdkit mol ###
index_list = ["33"]
ftype = "cry"
output = "../test/confs_local_minimization/data_QM.pt"
method = "QM"
datadir = "../test/confs_local_minimization/"
reference = "../test/reference_method/atomref.B3LYP_631Gd.10As.npz"
prepare_torch(index_list, output, datadir, reference, ftype,method)

In [6]:
### prepare target csv file for all targets after conversion from QM calculation ###
index_list = ["33"]
ftype = "cry"
output = "../test/confs_local_minimization/target.csv"
method = "QM"
datadir = "../test/confs_local_minimization/"
reference = "../test/reference_method/atomref.B3LYP_631Gd.10As.npz"
prepare_target_csv(index_list, output, datadir, reference, ftype)

In [7]:
### prepare the numpy file, which can be directly read into original PhysNet ###
index_list = ["33"]
datadir = "../test/confs_local_minimization/"
output = "../test/confs_local_minimization/PhysNet_input"
reference = "../test/reference_method/atomref.B3LYP_631Gd.10As.npz"
ftype = "cry"
prepre_PhysNet_input(index_list, output, datadir, reference, ftype, largest_num_atoms=44)

Here, we only implemented several basic data set prepartion functions, in the future, we can added more. 